In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.integrate
import scipy.special
import scipy.interpolate

import pydicom

In [ ]:
NE2571 = {
    'cavity': {
        'length': 24.0,
        'radius': 3.20
    },
    'electrode': {
        'length': 20.6,
        'radius': 1
    }
}

PTW30010 = {
    'cavity': {
        'length': 23.0,
        'radius': 3.05
    },
    'electrode': {
        'length': 20.6,
        'radius': 1
    }
}


chamber = NE2571

In [ ]:
DATA_DIR = Path(r"S:\Physics\RCCC Specific Files\Linac Beam Data Record\Synergy 2619\Absolute Dose Calibration\20191111 Absolute Calibration JM")

DATA_RESULTS_FILE = DATA_DIR.joinpath(f"2619_Measurements.xlsm")

data_results = pd.read_excel(DATA_RESULTS_FILE, sheet_name='6FFF_2').dropna()

In [ ]:
x = data_results['in-plane position'].values
profile_x = data_results['in-plane OAR'].values
y = data_results['cross-plane position'].values
profile_y = data_results['cross-plane OAR'].values

In [ ]:
xx, yy = np.meshgrid(x, y)

In [ ]:
def create_profile_func(x, profile_x, y, profile_y):
    x_interp = scipy.interpolate.interp1d(x, profile_x)
    y_interp = scipy.interpolate.interp1d(y, profile_y)
    
    def profile(xx, yy):
        return x_interp(xx) * y_interp(yy)
    
    return profile


profile = create_profile_func(x, profile_x, y, profile_y)

def weight_model_A(xx, yy):  
    result = np.ones_like(xx)
        
    return result

def weight_model_B(xx, yy):
    
    results = np.zeros_like(xx)
    assert np.shape(results) == np.shape(yy)
    
    
    results = np.sqrt(chamber['cavity']['radius']**2 - xx**2)
    
    return results

# def weight_model_C(xx, yy):
    
#     results = np.ones_like(xx) * np.nan
    
#     ref_1 = np.logical_and(
#         -chamber['cavity']['length']/2 <= yy,
#         yy < (chamber['electrode']['length'] - chamber['cavity']['length']/2),
#         xx <= chamber['cavity']['radius']
#     )
    
#     ref_2 = np.logical_and(
#        (chamber['electrode']['length'] - chamber['cavity']['length']/2) <= yy,
#         yy < chamber['cavity']['length']/2,
#         xx <= (chamber['cavity']['radius'] * ((yy-chamber['cavity']['length']/2)/(chamber['electrode']['length']-chamber['cavity']['length']))
#     )
    
#     results = np.sqrt(chamber['cavity']['radius']**2 - xx[ref_1]**2)
    
#     results = np.sqrt(((chamber['cavity']['radius']**2) * (((yy[ref_2]-chamber['cavity']['length']/2)**2)/(chamber['electrode']['length']-chamber['cavity']['length'])**2))- xx[ref_2]**2)
    
#     return results

weight = weight_model_A

def weighted_profile(xx, yy):
    return weight(xx, yy) * profile(xx, yy)

weight(xx,yy)

In [ ]:
 -chamber['cavity']['length']/2

In [ ]:
plt.contourf(xx, yy, profile(xx, yy), 50)
plt.axis('equal')
plt.colorbar()

In [ ]:
plt.contourf(xx, yy, weighted_profile(xx, yy), 100)
# plt.xlim([-5, 5])
# plt.ylim()

plt.colorbar()

In [ ]:
integral_bounds = [-chamber['cavity']['radius'], chamber['cavity']['radius']], [-chamber['cavity']['length']/2, chamber['cavity']['length']/2]

kvol_numerator = scipy.integrate.nquad(weight, integral_bounds)[0] 
kvol_denominator = scipy.integrate.nquad(weighted_profile, integral_bounds)[0]

kvol = kvol_numerator / kvol_denominator

kvol

In [ ]:
kvol_denominator

In [ ]:
plt.figure(1)

plt.subplot(111)
plt.plot(y,profile_y,label='inplane')
plt.plot(x,profile_x,label='crossplane')
plt.ylabel('Normalised OAR')
plt.xlabel('Plane Position (cm)')
plt.title('')
plt.legend(loc='lower left')


plt.subplots_adjust(top=3, bottom=0.5, left=0, right=2, hspace=0.25,
                    wspace=0.25)